In [2]:
import requests
import webbrowser
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
class Michelin:
    def __init__(self, page_num):
        self.page_num = page_num
        self.url = f'https://guide.michelin.com/gb/en/restaurants/page/{self.page_num}'
        #webbrowser.open(url)
        self.page = requests.get(self.url)
        self.soup = BeautifulSoup(self.page.text, 'html.parser')

    def find_elements(self):
        return self.soup.find_all('div', {"class": "card__menu-content js-match-height-content"})

    def get_details(self):
        elements = self.find_elements()
        restaurants_dict = {}
        for num in range(len(elements)):
            restaurant_name = elements[num].find('h3').getText().strip()
            location = elements[num].find('div', {"class": "card__menu-footer--location flex-fill pl-text"}).getText().strip()
            if ',' in location:
                city = location.split(",")[0]
                country = location.split(",")[1].strip()
            else:
                city = None
                country = location
                
            price = elements[num].find('div', {"class": "card__menu-footer--price pl-text"}).getText().strip().replace(" ",'').replace('·','').split()[0]
            
            if "·" in elements[num].find('div', {"class": "card__menu-footer--price pl-text"}).getText():
                cuisine = elements[num].find('div', {"class": "card__menu-footer--price pl-text"}).getText()[41:].replace('·','').strip()
            else:
                cuisine = elements[num].find('div', {"class": "card__menu-footer--price pl-text"}).getText().strip()
            images = elements[num].find_all('img')
            recognition = None
            
            if len(images) > 1:
                recognition = f'{len(images)} stars'
            elif len(images) == 1:
                if 'gourmand' in images[0]['src']:
                    recognition = 'Bib Gourmand'
                elif 'durable' in images[0]['src']:
                    recognition = 'Green Star'
                else:
                    recognition = '1 Star'
            else:
                recognition = None
                  
            restaurants_dict[restaurant_name] = [city, country, price, cuisine, recognition]
        return restaurants_dict

In [5]:

all_restaurants = {}
for number in range(1,828):
    michelin_page = Michelin(number)
    curr_dict = michelin_page.get_details()
    all_restaurants[number] = curr_dict

In [72]:
names = []
cities = []
countries = []
prices = []
styles = []
awards = []
for num in range(1,828):
    for name in all_restaurants[num]:
        city = all_restaurants[num][name][0]
        country = all_restaurants[num][name][1]
        price = all_restaurants[num][name][2]
        style = all_restaurants[num][name][3]
        award = all_restaurants[num][name][4]
        names.append(name)
        cities.append(city)
        countries.append(country)
        
        if len(price) > 4:
            prices.append(None)
        elif len(price) == 4:
            prices.append(4)
        elif len(price) == 3:
            prices.append(3)
        elif len(price) == 2:
            prices.append(2)
        else:
            prices.append(1)
        
        styles.append(style)
        awards.append(award)


In [73]:
restaurant_dictionary = {'Restaurant Name': names, 'City': cities, 'Country': countries, 'Price': prices, 'Style': styles, 'Awards': awards}
df = pd.DataFrame(data=restaurant_dictionary)


In [74]:
df.to_excel("restaurants3.xlsx",
             sheet_name='Sheet_name_1') 